ENVIRON

In [2]:
import os
import sys
from datetime import datetime
sys.path.insert(0, './boruvka-superpixel/pybuild')

In [1]:
from tqdm import tqdm, trange
from pandarallel import pandarallel
# import swifter
# from swifter import set_defaults
import skimage as ski
import numpy as np
import pandas as pd
from PIL import Image
from skimage import graph

In [3]:
import boruvka_superpixel
# from wavelet import utils_wavelet as wv
from skimage.segmentation import slic, felzenszwalb

In [4]:
from skimage import io, color

In [5]:
import networkx as nx

In [6]:
tqdm.pandas()
pandarallel.initialize(progress_bar = True, nb_workers = 8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


DATASET

In [7]:
images = []

In [8]:
for filename in os.listdir(f'./ImageNet_ILSVEC_2010_val'):
    images.append(f'./ImageNet_ILSVEC_2010_val/{filename}')

FileNotFoundError: [Errno 2] No such file or directory: './ImageNet_ILSVEC_2010_val'

In [9]:
df = pd.DataFrame(columns = ['filename', 'image', 'label'])

In [11]:
with open('./ILSVRC2010_validation_ground_truth.txt') as f:
    labels = f.readlines()
labels = [int(label.strip()) for label in labels]

In [12]:
print('BEFORE IMPORT:', str(datetime.now()))

BEFORE IMPORT: 2023-11-15 06:53:24.941166


In [13]:
for index, filename in enumerate(tqdm(images[0:1])):
    df.loc[index] = {'filename': filename, 'image': np.array(Image.open(filename).convert('RGB')), 'label': labels[index]}

100%|██████████| 1/1 [00:00<00:00, 86.96it/s]


In [14]:
print('AFTER IMPORT:', str(datetime.now()))

AFTER IMPORT: 2023-11-15 06:53:27.366062


SLIC - about 3 hours per 50k

In [15]:
def slic_conversion(image, num_sp):
    try:
        return slic(image, num_sp)
    except Exception as e:
        print(e)

In [16]:
print('BEFORE SLIC LABEL:', str(datetime.now()))

BEFORE SLIC LABEL: 2023-11-15 06:53:29.795064


In [17]:
df['slic_label'] = df['image'].parallel_apply(lambda x : slic_conversion(x, 75))

In [ ]:
df['slic_label'] = df['image'].progress_apply(lambda x : slic_conversion(x, 75))

In [ ]:
df['slic_label'] = df['image'].swifter.progress_bar(True).apply(lambda x : slic_conversion(x, 75))

In [ ]:
print('AFTER SLIC LABEL:', str(datetime.now()))

In [ ]:
print('BEFORE SLIC GRAPH:', str(datetime.now()))

In [ ]:
df['slic_graph'] = df.progress_apply(lambda x : graph.rag_mean_color(x['image'], x['slic_label']), axis = 1)

In [ ]:
print('AFTER SLIC GRAPH:', str(datetime.now()))

In [ ]:
image = np.zeros(df['image'][0].shape)

In [ ]:
df['image'][0] = color.rgb2lab(df['image'][0])

In [ ]:
Image.fromarray(df['image'][0].astype(np.uint8))

In [ ]:
df['image'][0].max()

In [ ]:
label = slic(df['image'][0], 75)

In [ ]:
G = graph.rag_mean_color(df['image'][0], label)

In [ ]:
ski.graph.show_rag(label, G, df['image'][0].astype(np.uint8))

BORUVKA SUPERPIXEL HIERARCHY - about 1 hour per 50k

In [ ]:
def boruvka_conversion(image, num_sp):
    try:
        image_edge = np.zeros((image.shape[:2]), dtype = image.dtype)

        bsp = boruvka_superpixel.BoruvkaSuperpixel()
        bsp.build_2d(image, image_edge)
        bsp_label = bsp.label(num_sp)

        return bsp_label
    except Exception as e:
        print(e)

In [ ]:
print('BEFORE BORUVKA LABEL:', str(datetime.now()))

In [ ]:
df['boruvka_label'] = df['image'].parallel_apply(lambda x : boruvka_conversion(x, 75))

In [ ]:
print('BEFORE BORUVKA LABEL:', str(datetime.now()))

In [ ]:
print('BEFORE BORUVKA GRAPH:', str(datetime.now()))

In [ ]:
G = graph.rag_mean_color(df['image'][0], df['boruvka_label'][0])

In [ ]:
bsp.average()

In [ ]:
image = df['image'][0]
image_edge = np.zeros((image.shape[:2]), dtype = image.dtype)

In [ ]:
bsp = boruvka_superpixel.BoruvkaSuperpixel()
bsp.build_2d(image, image_edge)
Image.fromarray(bsp.average(75, 3, image))

In [ ]:
df['boruvka_graph'] = df.parallel_apply(lambda x : graph.rag_mean_color(df['image'], df['boruvka_label']), axis = 1)

In [ ]:
print('AFTER BORUVKA GRAPH:', str(datetime.now()))

SAVE FOR TEMP

In [ ]:
df.to_csv('./imagenet50k.csv')